In [13]:
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
import sys
import os
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials


def get_vacancies(job_title="'Аналитик данных' and 'data analyst'", area=1, to_csv=False, table_name='hh', gs=False,your_key='some_key.json',my_mail='somemail@mail.com'):
    def getPage(page=0):

        params = {
            'text': job_title,
            'area': area,
            'page': page,
            'per_page': 100
        }

        req = requests.get('https://api.hh.ru/vacancies', params)
        data = req.content.decode()
        req.close()
        return data

    data = []
    for page in tqdm(range(0, 20), desc='Парсим по запросу '+job_title):
        jsObj = json.loads(getPage(page))
        data.append(jsObj)

    def highlighttext(column):
        return column.str.replace('[<\>\/]', '', regex=True).str.replace('highlighttext', '', regex=True)

    def google_exp():
        scope = ['https://spreadsheets.google.com/feeds',
                'https://www.googleapis.com/auth/drive']

       
        path_to_credentials = 'crdentials.json'

        # Authorization
        credentials = ServiceAccountCredentials.from_json_keyfile_name(
            your_key, scope)
        gs = gspread.authorize(credentials)

        # Create empty table
        table_name = 'HH'
        sheet = gs.create('HH')

        # Make it visible to other guys
        sheet.share(my_mail, perm_type='user', role='writer')

        spreadsheet_name = 'HH'
        sheet = 'HH'
        d2g.upload(df, table_name, sheet, credentials=credentials, row_names=True)
        

    df = pd.json_normalize(data, record_path=['items']).rename(
        columns=lambda c: c.replace('.', '_'))

    try:

        df = df[['id', 'name',
                'employer_name',
                 'apply_alternate_url',
                 'area_name',
                 'salary_from',
                 'salary_to',
                 'salary_currency',
                 'employer_url',
                 'snippet_requirement',
                 'snippet_responsibility',
                 'schedule_name', 'address_raw', 'address_lat',
                 'address_lng', 'address_metro_station_name']]

        df.snippet_responsibility = highlighttext(df.snippet_responsibility)
        df.snippet_requirement = highlighttext(df.snippet_requirement)
    except KeyError:
        print('Ошибка в запросе')
        sys.exit(0)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    #csv_name_with_timestamp = job_title+timestr+".csv"
    if to_csv == True:
        dir_name = 'hh_parsing_result'
        os.mkdir(dir_name)
        df.to_csv(dir_name + '/'+table_name+"_"+timestr+".csv", index=False)
        print('Сохранено как', table_name, 'в', pd.Timestamp.now())
    else:
        pass
    if gs == True:
        google_exp()
        print('Сохранено в Google Sheets'+'', pd.Timestamp.now())

    else:
        pass
        print('Выполнено в'+'', pd.Timestamp.now())
    return df


In [14]:
df1=get_vacancies(job_title="'Аналитик данных' and 'data analyst'",
              area=1,to_csv=True,table_name='hhtest', gs=False,your_key='some_key.json',my_mail='somemail@mail.com')


Парсим по запросу 'Аналитик данных' and 'data analyst': 100%|██████████| 20/20 [00:15<00:00,  1.29it/s]

Сохранено как hhtest в 2022-11-01 11:16:26.565343
Выполнено в 2022-11-01 11:16:26.565495


In [15]:
df1

,id,name,employer_name,apply_alternate_url,area_name,salary_from,salary_to,salary_currency,employer_url,snippet_requirement,snippet_responsibility,schedule_name,address_raw,address_lat,address_lng,address_metro_station_name
0,71233070,Аналитик данных / Data Analyst,Можайская executive,https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,140000.0,RUR,https://api.hh.ru/employers/198875,Git – как преимущество. Также будет преимущест...,...данных (DWH). Наращивание функциональности ...,Полный день,NaN,NaN,NaN,NaN
1,71424469,Аналитик данных,билайн,https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,NaN,NaN,https://api.hh.ru/employers/4934,Опыт в анализе данных (продуктовом финансовом)...,Заниматься сбором информации и подготовкой отч...,Удаленная работа,NaN,NaN,NaN,NaN
2,71562226,Data Analyst,Stellar Labs,https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,NaN,NaN,https://api.hh.ru/employers/4176223,"Умение четко, структурированно и убедительно п...","...подключать источники данных, работать с API...",Полный день,"Москва, Лесная улица, 43",55.780588,37.590677,Белорусская
3,70686241,Data-аналитик,билайн,https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,NaN,NaN,https://api.hh.ru/employers/4934,Опыт работы аналитиком данных от 2 лет. Уверен...,Исследование источников данных. Взаимодействие...,Полный день,"Москва, Краснопролетарская улица",55.776862,37.607008,Новослободская
4,69981940,Аналитик данных,АЙТИ.СПЕЙС,https://hh.ru/applicant/vacancy_response?vacan...,Москва,200000.0,350000.0,RUR,https://api.hh.ru/employers/2000762,Опыт работы на проектах по внедрению развитию...,Описание модели данных и полей данных из источ...,Полный день,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869,55567933,Лид аналитики (секретарь Олег),Тинькофф,https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,NaN,NaN,https://api.hh.ru/employers/78638,"...опыта в качестве аналитика. Умения строить,...",Продвигать data-driven культуру при принятии п...,Полный день,"Москва, Головинское шоссе, 5",55.840240,37.492024,Водный стадион
870,70691838,Product manager B2C,Работа.ру,https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,NaN,NaN,https://api.hh.ru/employers/1870,Умеешь обосновывать принимаемые продуктовые ре...,Формировать стратегию и vision своего продукто...,Полный день,"Москва, 3-я улица Ямского Поля, 9",55.781626,37.580894,NaN
871,70678733,"Главный эксперт, Автоматизация процессов (Охра...","СИБУР, Группа компаний",https://hh.ru/applicant/vacancy_response?vacan...,Москва,NaN,NaN,NaN,https://api.hh.ru/employers/3809,"Умеете работать с данными (data mamagement, da...",Самостоятельной разработкой и поддержкой инстр...,Полный день,None,NaN,NaN,Профсоюзная
872,71040345,QA Automation Engineer (релокация на Кипр),Soveren,https://hh.ru/applicant/vacancy_response?vacan...,Москва,3000.0,5500.0,USD,https://api.hh.ru/employers/5200755,Навык разработки frontend и backend. Опыт разр...,Создание и запуск автоматизированных тестов дл...,Удаленная работа,NaN,NaN,NaN,NaN
